In [ ]:
# install verdict
!uv pip install verdict --system

# This notebook has been run ahead of time, so you can inspect outputs without making
# any API calls. You can set your API key if you want to run the examples yourself.
# %env OPENAI_API_KEY=*************************

> [**LLM Evaluators Recognize and Favor Their Own Generations**](https://arxiv.org/abs/2404.13076)  
> Arjun Panickssery, Samuel R. Bowman, Shi Feng  
> NeurIPS 2024

In [58]:
from datasets import load_dataset
from verdict.dataset import DatasetWrapper

ds = load_dataset("EdinburghNLP/xsum")

dataset = DatasetWrapper.from_hf(
    ds, 
    columns=["document"],
    max_samples=50
)

In [59]:
from verdict import Unit
from verdict.schema import Schema

class SummarizerUnit(Unit):
    class ResponseSchema(Schema):
        summary: str

In [60]:
from verdict import Pipeline, Layer
from verdict.common.judge import PairwiseJudgeUnit
from verdict.transform import MapUnit
from verdict.extractor import SampleScoreExtractor

models = ["gpt-3.5-turbo", "gpt-4"]
pipeline = Pipeline("Self-Preference") \
    >> Layer([
        SummarizerUnit().prompt("""
            Article:
            {source.document}

            Provide a one-sentence summary (no more than 30 words) with no other text.

            @system
            You are a news-article summarizer. Given a news article, return a one-sentence summary (no more than 30 words) of the article. This will really help us better understand the article.
            Return only the one-sentence summary with no other text.
        """).via(model, temperature=0.0) for model in models
    ]) \
    >> Layer([
        PairwiseJudgeUnit().prompt("""
        Pick the better summary for the following reference article. Respond with either 'A' or 'B'.

        Article:
        {source.document}

        A: {previous.summarizer[0].summary}
        B: {previous.summarizer[1].summary}
        """).via(model) for model in models
    ])

In [61]:
df, _ = pipeline.run_from_dataset(dataset['test'], max_workers=1024)

In [62]:
for i, model in enumerate(models):
    print(f"{model} self-preference: {(df[f'Self-Preference_root.block.block.layer[{i}].unit[PairwiseJudge]_choice'] == 'A').mean()}")

gpt-3.5-turbo self-preference: 0.98
gpt-4 self-preference: 0.38
